In [52]:
import re
import numpy as np
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from pymystem3 import Mystem
from collections import defaultdict, Counter
from itertools import combinations
import json

%matplotlib inline
import matplotlib.pyplot as plt

In [162]:
m = Mystem(entire_input=False)

# Read data

In [3]:
with open('seed.txt','r',encoding='utf-8') as f:
    gs = [x.strip() for x in f.readlines()]

In [4]:
# lemmatized (mystem) texts
with open('lemmas.txt','r',encoding='utf-8') as f:
    lemmas = json.loads(f.read())

In [218]:
print(len(lemmas))
print(sum([len(x) for x in lemmas]))

19034
3022287


# Word2vec

In [150]:
model = Word2Vec(lemmas, size=300, window=7, min_count=3)

In [163]:
# find most similar - 2 levels

def w2v_crawl_shrinked(word):
    all_words = set()
    lemma = m.lemmatize(word)
    if len(lemma) == 1:
        lemma = lemma[0]
        if lemma in model:
            sim1 = {x[0] for x in model.wv.most_similar(lemma) if model.similarity(lemma,x[0]) >= 0.6}
            all_words |= sim1
            for w in sim1:
                sim2 = {x[0] for x in model.wv.most_similar(w) if model.similarity(lemma,x[0]) >= 0.6}
                all_words |= sim2
    return all_words

In [164]:
all_words = set()
for word in gs:
    curr_words = w2v_crawl_shrinked(word)
    print(word,len(curr_words))
    all_words |= curr_words

аналогов нет 0
подошва 72
вежливый 15
великолепный 24
вкусный 6
внимательный 19
большой выбор 0
вау 79
неплохой 15
идеальный 11
неприятный 0
понравилось 6
рекомендуем 2
на ура 0
советуем 2
отвратительный 12
отличный 20
положительный 13
отрицательный 21
приятный 11
напоминать о себе 0
спасибо 13
радовать глаз 0
на совесть 0
молодцы 38
придти еще раз 0
орать 46
улыбаться 39
приветливый 16
отличный 20
замечательный 21


# Topic modelling

In [202]:
vectorizer = TfidfVectorizer(stop_words=stopwords.words('russian'), min_df = 10)
A = vectorizer.fit_transform([' '.join(x) for x in lemmas])
terms = vectorizer.get_feature_names()

In [32]:
# show topic descriptors
def get_descriptor( terms, H, topic_index, top ):
    top_indices = np.argsort( H[topic_index,:] )[::-1]
    top_terms = []
    for term_index in top_indices[0:top]:
        top_terms.append( terms[term_index] )
    return top_terms

def get_all_descriptors(k, H, terms, top):
    for topic_index in range(k):
        descriptor = get_descriptor( terms, H, topic_index, top )
        str_descriptor = ", ".join( descriptor )
        print("Topic %02d: %s" % ( topic_index+1, str_descriptor ) )

In [203]:
k = 20

nmf = NMF( init="nndsvd", n_components=k, random_state=42 ) 
W = nmf.fit_transform( A ) # тематическое представление тектов
H = nmf.components_  # темы с вероятностями слов быть по ним

get_all_descriptors(k, H, terms, 10)

Topic 01: салат, соус, суп, блюдо, вкус, мясо, десерт, меню, овощ, тарелка
Topic 02: quot, вопрос, название, цезарь, слово, сегодня, ответ, официантка, который, спрашивать
Topic 03: понравиться, очень, особенно, обязательно, прийти, еда, интерьер, зайти, друг, подруга
Topic 04: свадьба, спасибо, гость, наш, весь, банкет, праздник, зал, огромный, оставаться
Topic 05: это, мочь, место, весь, сказать, который, человек, свой, стол, зал
Topic 06: приносить, минута, заказ, ждать, официант, заказывать, столик, официантка, счет, час
Topic 07: ресторан, блюдо, посещать, любимый, ваш, отзыв, год, повар, это, официант
Topic 08: пиво, паб, пивной, сорт, закуска, бар, попить, футбол, гренок, колбаска
Topic 09: отличный, место, рекомендовать, советовать, сервис, кухня, друг, замечательный, уютно, отдых
Topic 10: очень, вкусно, уютно, быстро, вкусный, довольный, приятно, оставаться, красиво, большой
Topic 11: кухня, обслуживание, интерьер, уровень, высокий, неплохой, официант, замечательный, мочь, се

In [204]:
gs_vects = [vectorizer.transform([' '.join(m.lemmatize(x))]) for x in gs] # create "docs" from gold standard

In [210]:
topics = []
for i,v in enumerate(gs_vects):
    W = nmf.transform(v)
    if sum(W[0]):
        print(gs[i],np.argsort(W[0])[::-1][:5])
        topics.append(np.argsort(W[0])[::-1][:5])
sent_topics = [x[0] for x in Counter([y for x in topics for y in x]).most_common(10)]
sent_topics

аналогов нет [ 0 17 10 13 14]
подошва [15  5  0  7  8]
вежливый [11  9  2 12 15]
великолепный [10 11  3  6  8]
вкусный [11 16 14  7  9]
внимательный [11  9  3 10 12]
большой выбор [15  3  7  0  9]
вау [17  0 18  1 19]
неплохой [10  7 15 14 13]
идеальный [ 0 11 17 10 15]
неприятный [ 5 13  4  6 10]
понравилось [ 2 19 18  1  3]
рекомендуем [ 8 17 15 11  6]
на ура [ 3 18 12 17  7]
советуем [ 8 19 18 10  9]
отвратительный [ 5 16 10 13 18]
отличный [ 8 19 18  1  2]
положительный [13 11  3 10  6]
отрицательный [ 4  6  1 15  5]
приятный [11 19  8  1  2]
напоминать о себе [ 0  5  4  1 13]
спасибо [ 3 11 18 19  8]
радовать глаз [ 4 19 15 11  9]
на совесть [ 4  3  0 12  8]
молодцы [18  3  8  9  2]
придти еще раз [ 2 11 18 12  5]
орать [ 4 14 11  5 12]
улыбаться [ 4  5 11  0  9]
приветливый [11 13  9 19 12]
отличный [ 8 19 18  1  2]
замечательный [11 10 18  3  8]


[11, 8, 18, 3, 10, 9, 19, 0, 5, 15]

In [206]:
all_words = sorted(list(all_words))
all_words_vects = [vectorizer.transform([x]) for x in all_words]

In [207]:
new_words = []
waste = []
for i,v in enumerate(all_words_vects):
    W = nmf.transform(v)
    if sum(W[0]):
        ts = np.argsort(W[0])[::-1][:6]
        if sum([x in sent_topics for x in ts]) > 3:
            new_words.append(all_words[i])
        else:
            waste.append(all_words[i])
    else:
        waste.append(all_words[i])

In [208]:
brand_new_words = [x for x in new_words if x not in gs]
len(brand_new_words)

79

In [209]:
with open('sentiment_words.txt','w',encoding='utf-8-sig') as f:
    f.write('\n'.join(brand_new_words))

# Evaluation

In [214]:
with open('rusentilex_2017.txt','r',encoding='utf-8') as f:
    senti_words = {x.split(',')[0] for x in f.readlines()}

in_dict = sum([x in senti_words for x in brand_new_words])
total = len(brand_new_words)
print(in_dict,total-in_dict,total,in_dict/total)

33 46 79 0.4177215189873418


In [220]:
for x in brand_new_words:
    if x in senti_words:
        print(x)

безобразный
безупречный
бесподобный
восхитительный
грамотный
доброжелательный
добротный
заботливый
изумительный
интересный
любезный
милый
навязчивый
объедение
отвратный
отзывчивый
отменный
потертость
потрясать
потрясающий
превосходный
предупредительный
прекрасный
пресный
приемлемый
профессиональный
симпатичный
смеяться
ужасный
улыбчивый
уютный
чудесный
шикарный


In [215]:
for x in brand_new_words:
    if x not in senti_words:
        print(x)

out
андрей
барбареско
благодарить
великолепно
вкусно
гардеробщик
громкость
девушка
жеваться
жир
завуалировать
зажигательный
зажигать
замечательно
картошка
маринованный
молодец
негромкий
ненавязчивый
обалденный
обслуживающий
общаться
ольга
отблагодарить
отвратительно
перекрикивать
поздороваться
порадовать
приветливо
разговаривать
раздевать
разочаровывать
ребята
рекомендовать
слушать
смородиновый
собеседник
советовать
удручающий
улыбка
умничек
услужливый
учтивый
шутить
этнический
